### 1-Strength Build Optimization: Class Selection, Stat Allocation, and Weapon Recommendation


Let's solve the Strength build optimization with data!

Given a target level of 150, we want to determine:

- Which starting class a hero should pick.
- How their stats should be allocated to maximize Strength efficiency while respecting key constraints.
- Which weapons will make the build truly devastating, considering only the **highest total attack version** per weapon that the hero can actually equip.

Constraints / Considerations:

1- **Strength**: primary stat, aim up to **80 points**.  
2- **Vigor**: secondary stat for survivability, allocate up to **60 points**.  
3- **Endurance**: secondary stat affecting stamina and equip load, allocate up to **50 points**.  
4- **Other stats** (Mind, Dexterity, Intelligence, Faith, Arcane): generally not increased for pure Strength builds; points here are wasted.  
5- **Minimize wasted points**: allocate as much as possible to Strength, Vigor, and Endurance only.  
6- **Weapons**: choose only those with **S or A Strength scaling**, **within the hero’s Strength, Dexterity, Intelligence, Faith, and Arcane limits**, and only the **highest upgrade per weapon**.

> Note: With only 10 starting classes we could eyeball the hero class, but using this optimizer scales effortlessly if the number of classes is very large.


In [0]:
%sql
SELECT * FROM eldenringcatalog.gold.startingclasses

Class_Name,Level,Vigor,Mind,Endurance,Strength,Dexterity,Intelligence,Faith,Arcane
Vagabond,9,15,10,11,14,13,9,9,7
Warrior,8,11,12,11,10,16,10,8,9
Hero,7,14,9,12,16,9,7,8,11
Bandit,5,10,11,10,9,13,9,8,14
Astrologer,6,9,15,9,8,12,16,7,9
Prophet,7,10,14,8,11,10,7,16,10
Samurai,9,12,11,13,12,15,9,8,8
Prisoner,9,11,12,11,11,14,14,6,9
Confessor,10,10,13,10,12,12,9,14,9
Wretch,1,10,10,10,10,10,10,10,10



Elden Ring offers 10 starting classes. Each class begins at a specific level and comes with its own distribution of attributes such as Vigor, Mind, Endurance, Strength, Dexterity, Intelligence, Faith, and Arcane.


Let’s build the StrengthBuildOptimizer class to tackle our problem: given a target level of 150, it will determine which starting class our hero should pick, allocate stats to maximize Strength efficiency while respecting key constraints, minimize wasted points in irrelevant attributes, and recommend the highest upgrade version of weapons with S or A Strength scaling that the hero can equip based on Strength, Dexterity, Intelligence, Faith, and Arcane stats

#### **StrengthBuildOptimizer class**

In [0]:
import logging
logger = logging.getLogger(__name__)

class StrengthBuildOptimizer:

    def __init__(self, spark):
        self.spark = spark

    # Load starting classes
    def _load_classes(self):
        df = self.spark.sql("SELECT * FROM eldenringcatalog.gold.startingclasses")
        # Normalize columns to lowercase
        for col in df.columns:
            df = df.withColumnRenamed(col, col.lower())
        return df.collect()

    # Choose best starting class for PURE STR build
    def get_best_str_class(self):
        classes = self._load_classes()

        useful = ["strength", "vigor", "endurance"]
        ranked = []

        for c in classes:
            base_stats = {
                "vigor": c.vigor,
                "mind": c.mind,
                "endurance": c.endurance,
                "strength": c.strength,
                "dexterity": c.dexterity,
                "intelligence": c.intelligence,
                "faith": c.faith,
                "arcane": c.arcane
            }

            useful_sum = sum(base_stats[s] for s in useful)
            wasted_sum = sum(v for k, v in base_stats.items() if k not in useful)
            efficiency = useful_sum - wasted_sum

            ranked.append({
                "class_name": c.class_name,
                "starting_level": c.level,
                "base_stats": base_stats,
                "efficiency_score": efficiency
            })

        best = sorted(ranked, key=lambda r: r["efficiency_score"], reverse=True)[0]
        logger.info(f"🏆 Best class for PURE STR = {best['class_name']}")
        return best

    # Stat allocation for PURE STR ONLY
    def allocate_str_stats(self, base, available_points):

        soft_caps = {
            "vigor": 60,
            "endurance": 50,
            "strength": 80
        }

        ratios = {
            "strength": 0.50,
            "vigor": 0.30,
            "endurance": 0.20
        }

        stats = base.copy()

        for stat, pct in ratios.items():
            stats[stat] = int(min(soft_caps[stat], stats[stat] + available_points * pct))

        return stats


    # FINAL PURE STR BUILD OPTIMIZATION
    def optimize_pure_str_build(self, target_level: int = 150):

        # 1) Best STR class
        best_class = self.get_best_str_class()
        base_stats = best_class["base_stats"]
        starting_level = best_class["starting_level"]
        available_points = target_level - starting_level

        # 2) Allocate stats
        final_stats = self.allocate_str_stats(base_stats, available_points)

        # 3) Recommended STR weapons
        df_items = self.spark.sql(f"""
                        SELECT weapon_name, category, upgrade_level, total_attack_power,
                            attack_physical, attack_magic, attack_fire, attack_lightning, attack_holy,
                            scaling_strength, scaling_dexterity
                        FROM (
                            SELECT w.weapon_name, w.category, ws.upgrade_level, ws.total_attack_power,
                                ws.attack_physical, ws.attack_magic, ws.attack_fire, ws.attack_lightning, ws.attack_holy,
                                ws.scaling_strength, ws.scaling_dexterity,
                                ROW_NUMBER() OVER (PARTITION BY w.weapon_name ORDER BY ws.total_attack_power DESC) as rn
                            FROM eldenringcatalog.gold.dim_weapons w
                            JOIN eldenringcatalog.gold.fact_weapon_stats ws
                                ON w.weapon_name = ws.weapon_name
                            WHERE ws.scaling_strength IN ('S', 'A')
                            AND w.required_strength <= {final_stats['strength']}
                            AND w.required_intelligence <= {final_stats['intelligence']}
                            AND w.required_faith <= {final_stats['faith']}
                            AND w.required_arcane <= {final_stats['dexterity']}
                            AND w.required_dexterity <= {final_stats['dexterity']}
                            AND w.is_dlc = 0

                        ) t
                        WHERE rn = 1
                        ORDER BY total_attack_power DESC
                        LIMIT 10
                    """)

        return {
            "target_level": target_level,
            "starting_class": best_class["class_name"],
            "starting_level": starting_level,
            "base_stats": base_stats,
            "optimized_stats": final_stats,
            "recommended_items": df_items
        }


In [0]:
optimizer = StrengthBuildOptimizer(spark)
result = optimizer.optimize_pure_str_build(target_level=150)

# Display summary
print(f"\n🏹 Strength Build Summary (Target Level: {result['target_level']})")
print(f"Starting Class: {result['starting_class']} (Level {result['starting_level']})\n")

# Base stats
print("Base Stats:")
for stat, value in result['base_stats'].items():
    print(f"  {stat.capitalize():<12}: {value}")

# Optimized stats
print("\nOptimized Stats:")
for stat, value in result['optimized_stats'].items():
    print(f"  {stat.capitalize():<12}: {value}")

# Recommended weapons
print("\nTop Recommended STR Weapons:")
display(result['recommended_items'])


🏹 Strength Build Summary (Target Level: 150)
Starting Class: Hero (Level 7)

Base Stats:
  Vigor       : 14
  Mind        : 9
  Endurance   : 12
  Strength    : 16
  Dexterity   : 9
  Intelligence: 7
  Faith       : 8
  Arcane      : 11

Optimized Stats:
  Vigor       : 56
  Mind        : 9
  Endurance   : 40
  Strength    : 80
  Dexterity   : 9
  Intelligence: 7
  Faith       : 8
  Arcane      : 11

Top Recommended STR Weapons:


weapon_name,category,upgrade_level,total_attack_power,attack_physical,attack_magic,attack_fire,attack_lightning,attack_holy,scaling_strength,scaling_dexterity
Giant-Crusher,Colossal Weapons,Cold +25,473.0,270.0,203.0,0.0,0.0,0.0,A,-
Warped Axe,Axes,Cold +25,378.0,216.0,162.0,0.0,0.0,0.0,A,-
Gargoyle's Great Axe,Greataxes,Cold +25,374.0,214.0,160.0,0.0,0.0,0.0,A,E
Brick Hammer,Great Hammers,Cold +25,372.0,212.0,160.0,0.0,0.0,0.0,A,-
Greataxe,Greataxes,Heavy +25,343.0,343.0,0.0,0.0,0.0,0.0,A,-
Prelate's Inferno Crozier,Colossal Weapons,Heavy +25,332.0,332.0,0.0,0.0,0.0,0.0,A,-
Longhaft Axe,Greataxes,Heavy +25,331.0,331.0,0.0,0.0,0.0,0.0,A,-
Executioner's Greataxe,Greataxes,Heavy +25,319.0,319.0,0.0,0.0,0.0,0.0,A,-
Rusted Anchor,Greataxes,Heavy +25,312.0,312.0,0.0,0.0,0.0,0.0,A,-
Battle Axe,Axes,Heavy +25,286.0,286.0,0.0,0.0,0.0,0.0,A,-


# 2-What weapon should I choose at level 1 that will still be viable at level 150 (Strenght Build) ?

From our analysis, the **Hero** class emerges as the best starting choice for a Strength build, as it minimizes wasted stat points.  

**Base stats for the Hero class:**

- Vigor       : 14  
- Mind        : 9  
- Endurance   : 12  
- Strength    : 16  
- Dexterity   : 9  
- Intelligence: 7  
- Faith       : 8  
- Arcane      : 11  

**Optimized stats at target level 150:**

- Vigor       : 56  
- Mind        : 9  
- Endurance   : 40  
- Strength    : 80  
- Dexterity   : 9  
- Intelligence: 7  
- Faith       : 8  
- Arcane      : 11


so level-1 weapon constraints become 
required_strength  <= 16 &
required_dexterity <= 9

at level 150 with strength build :
Strength : 80 and Dexterity : 9

To remain strong late game, the weapon must have: scaling_strength IN ('A', 'S')


In [0]:
%sql
-- Step 1: Start from hero class stats
WITH hero_stats AS (
    SELECT 16 AS hero_strength, 9 AS hero_dexterity
)

SELECT 
    w.weapon_id,
    w.weapon_name,
    w.required_strength,
    w.required_dexterity,
    fs.upgrade_level,
    fs.scaling_strength,
    fs.total_attack_power,
    fs.attack_physical,
    fs.attack_magic,
    fs.attack_fire,
    fs.attack_lightning
FROM eldenringcatalog.gold.dim_weapons w
CROSS JOIN hero_stats h
JOIN eldenringcatalog.gold.fact_weapon_stats fs 
    ON w.weapon_name = fs.weapon_name
WHERE 
    -- Hero can equip at level 1
    w.required_strength <= h.hero_strength
    AND w.required_dexterity <= h.hero_dexterity

    -- Good endgame scaling
    AND fs.scaling_strength IN ('A', 'S')
    AND w.is_dlc = 0

    -- Ensure we select the MAX upgrade level version
    AND fs.total_attack_power = (
    SELECT MAX(fs2.total_attack_power)
    FROM eldenringcatalog.gold.fact_weapon_stats fs2
    WHERE fs2.weapon_name = w.weapon_name
)

ORDER BY 
    fs.scaling_strength DESC,   -- S first
    w.required_strength ASC;    -- lower requirements preferred


weapon_id,weapon_name,required_strength,required_dexterity,upgrade_level,scaling_strength,total_attack_power,attack_physical,attack_magic,attack_fire,attack_lightning
278,Serpent-Hunter,0,0,Standard +10,A,271.0,271.0,0.0,0.0,0.0


Serpent-Hunter cannot be obtained at the beginning of the game. It is only found on a corpse to the left as you enter Rykard, Lord of Blasphemy’s boss arena. As a result, its required Strength and Dexterity values should not be considered for early-game analysis, and the data currently associated with Serpent-Hunter appears to be incorrect and needs to be corrected.
Given these constraints, **Lamenting Visage** remains the only valid candidate that satisfies our criteria.

![Lamenting Visage](https://static0.gamerantimages.com/wordpress/wp-content/uploads/2024/07/elden-ring-lamenting-visage-stats.jpg?q=49&fit=contain&w=750&h=422&dpr=2)

# 3-How should I spend my levels so I have enough HP, stamina, and weapon power to not die instantly in the Margit fight for a pure Strength build?

Margit hits extremely hard and very early in the game. 

Let’s optimize the first 20 levels for a pure Strength build to survive Margit. We want to determine:

- The best starting class that maximizes Strength efficiency while keeping Vigor and Endurance at survivable levels.
- How to allocate attribute points for the first 20 levels, prioritizing survival and sufficient Strength to wield early-game weapons.
- Which weapons to use, considering only early-game obtainable upgrades (e.g., +0 to +3) that fit the hero’s Strength and Dexterity and are not DLC.

Constraints / Considerations:

1- **Strength**: primary stat, allocate remaining points after survival thresholds.

2- **Vigor**: ensure enough HP to survive Margit (target ~25).

3- **Endurance**: enough stamina to roll, block, and swing weapons (~15).

4- **Other stats** (Mind, Dexterity, Intelligence, Faith, Arcane): generally not increased; points here are wasted.

5- **Weapons**: only those with S or A Strength scaling, early-game reachable upgrades (+0 to +3), and non-DLC.


In [0]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.window import Window

logger = logging.getLogger(__name__)

class StrengthEarlyGameOptimizer:
    """
    Optimize the first 20 levels for a pure Strength build to survive Margit,
    using Elden Ring tables: startingclasses, dim_weapons, fact_weapon_stats.
    
    Early-game weapon constraints:
    - Only weapons with +0 to +3 upgrades (no number is treated as +0)
    - Only weapons with Strength scaling 'S' or 'A'
    - Only non-DLC weapons (is_dlc = 0)
    """

    def __init__(self, spark, target_level=20):
        self.spark = spark
        self.target_level = target_level

    # Load starting classes
    def _load_classes(self):
        df = self.spark.sql("SELECT * FROM eldenringcatalog.gold.startingclasses")
        for col in df.columns:
            df = df.withColumnRenamed(col, col.lower())
        return df.collect()

    # Pick best starting class for STR build
    def get_best_str_class(self):
        classes = self._load_classes()
        useful = ["strength", "vigor", "endurance"]
        ranked = []

        for c in classes:
            base_stats = {
                "vigor": c.vigor,
                "mind": c.mind,
                "endurance": c.endurance,
                "strength": c.strength,
                "dexterity": c.dexterity,
                "intelligence": c.intelligence,
                "faith": c.faith,
                "arcane": c.arcane
            }
            useful_sum = sum(base_stats[s] for s in useful)
            wasted_sum = sum(v for k, v in base_stats.items() if k not in useful)
            efficiency = useful_sum - wasted_sum
            ranked.append({
                "class_name": c.class_name,
                "starting_level": c.level,
                "base_stats": base_stats,
                "efficiency_score": efficiency
            })

        best = sorted(ranked, key=lambda r: r["efficiency_score"], reverse=True)[0]
        logger.info(f"🏆 Best class for STR early-game = {best['class_name']}")
        return best

    # Allocate first N levels (here 20) based on STR build priorities
    def allocate_first_levels(self, base_stats):
        """
        Allocate points:
        1. Vigor to survive (~25)
        2. Endurance for stamina (~15)
        3. Remaining points -> Strength
        """
        points_to_spend = self.target_level - base_stats.get("level", 1)

        thresholds = {
            "vigor": 25,
            "endurance": 15,
            "strength": 25
        }

        allocation = base_stats.copy()

        # 1) Increase Vigor
        vigor_up = min(points_to_spend, max(0, thresholds["vigor"] - allocation["vigor"]))
        allocation["vigor"] += vigor_up
        points_to_spend -= vigor_up

        # 2) Increase Endurance
        end_up = min(points_to_spend, max(0, thresholds["endurance"] - allocation["endurance"]))
        allocation["endurance"] += end_up
        points_to_spend -= end_up

        # 3) Remaining points -> Strength
        allocation["strength"] += points_to_spend
        points_to_spend = 0

        return allocation

    # Recommend weapons the hero can wield
    def recommend_weapons(self, final_stats):
        df = self.spark.sql(f"""
            SELECT w.weapon_name, w.category, ws.upgrade_level, ws.total_attack_power,ws.attack_physical,ws.attack_magic,ws.attack_fire,ws.attack_lightning,ws.scaling_strength, ws.scaling_dexterity, w.is_dlc
            
            FROM eldenringcatalog.gold.dim_weapons w
            JOIN eldenringcatalog.gold.fact_weapon_stats ws
                ON w.weapon_name = ws.weapon_name
            WHERE w.is_dlc = 0
              AND w.required_strength <= {final_stats['strength']}
              AND w.required_dexterity <= {final_stats['dexterity']}
              AND w.required_intelligence <= {final_stats['intelligence']}
              AND w.required_faith <= {final_stats['faith']}
              AND w.required_arcane <= {final_stats['arcane']}
            
            ORDER by ws.total_attack_power DESC
  
        """)

        # Extract upgrade number, default 0 if no number
        df = df.withColumn(
            "upgrade_number",
            F.when(F.col("upgrade_level").contains("+"),
                   F.regexp_extract(F.col("upgrade_level"), r"\+([0-9]+)", 1).cast("int")
            ).otherwise(F.lit(0))
        )

        # Filter for early-game upgrades: +0 to +3
        df = df.filter(F.col("upgrade_number") <= 3)

        # Take the best upgrade per weapon (highest total_attack_power)
        window = Window.partitionBy("weapon_name").orderBy(F.col("total_attack_power").desc())
        df = df.withColumn("rn", F.row_number().over(window)).filter(F.col("rn") == 1).drop("rn", "upgrade_number")

        return df

    # Full optimization: first 20 levels
    def optimize_first_20_levels(self):
        best_class = self.get_best_str_class()
        base_stats = best_class["base_stats"]
        base_stats["level"] = best_class["starting_level"]

        final_stats = self.allocate_first_levels(base_stats)
        weapons = self.recommend_weapons(final_stats)

        return {
            "starting_class": best_class["class_name"],
            "base_stats": base_stats,
            "final_stats_level_20": final_stats,
            "recommended_weapons": weapons
        }



In [0]:
optimizer = StrengthEarlyGameOptimizer(spark, target_level=20)

# Run the optimization
results = optimizer.optimize_first_20_levels()

# Print starting class
print("🏹 Best Starting Class for STR Early Game:", results["starting_class"])

# Print base stats
print("\nBase Stats:")
for stat, value in results["base_stats"].items():
    print(f"  {stat}: {value}")

# Print final stats after first 20 levels
print("\nFinal Stats at Level 20:")
for stat, value in results["final_stats_level_20"].items():
    print(f"  {stat}: {value}")

# Show recommended early-game weapons
print("\nRecommended Weapons (Early Game, STR scaling, +0 to +3 upgrades):")
results["recommended_weapons"].display()

🏹 Best Starting Class for STR Early Game: Hero

Base Stats:
  vigor: 14
  mind: 9
  endurance: 12
  strength: 16
  dexterity: 9
  intelligence: 7
  faith: 8
  arcane: 11
  level: 7

Final Stats at Level 20:
  vigor: 25
  mind: 9
  endurance: 14
  strength: 16
  dexterity: 9
  intelligence: 7
  faith: 8
  arcane: 11
  level: 7

Recommended Weapons (Early Game, STR scaling, +0 to +3 upgrades):


weapon_name,category,upgrade_level,total_attack_power,attack_physical,attack_magic,attack_fire,attack_lightning,scaling_strength,scaling_dexterity,is_dlc
Battle Axe,Axes,Flame+3,240.0,120.0,0.0,120.0,0.0,E,E,0
Beast-Repellent Torch,Torches,Standard +3,190.0,68.0,0.0,122.0,0.0,C,E,0
Caestus,Fists,Flame+3,168.0,84.0,0.0,84.0,0.0,E,E,0
Celebrant's Cleaver,Axes,Flame+3,236.0,118.0,0.0,118.0,0.0,E,E,0
Club,Hammers,Flame+3,192.0,96.0,0.0,96.0,0.0,E,-,0
Curved Club,Hammers,Flame+3,214.0,107.0,0.0,107.0,0.0,E,E,0
Dagger,Daggers,Flame+3,140.0,70.0,0.0,70.0,0.0,E,E,0
Frenzied Flame Seal,Sacred Seals,Standard +3,35.0,35.0,0.0,0.0,0.0,D,D,0
Hand Axe,Axes,Flame+3,220.0,110.0,0.0,110.0,0.0,E,E,0
Harp Bow,Light Bows,Standard +3,88.0,88.0,0.0,0.0,0.0,E,E,0


To survive Margit early in the game with a pure Strength build, start as the Hero and focus your level-ups on Vigor, Endurance, and Strength. Raise Vigor to around 25 to ensure enough HP to withstand Margit’s attacks, and increase Endurance to about 14 to have sufficient stamina for rolling, blocking, and swinging your Strength weapons. Allocate the remaining points to Strength to wield high-damage early-game weapons effectively, while leaving other stats (Dexterity, Mind, Intelligence, Faith, Arcane) untouched, as points there are wasted. Equip Strength-scaling weapons with upgrades from +0 to +3 such as Battle Axe, Hand Axe, Stone Club, or Spiked Club to maximize early-game damage without relying on high-level upgrades. This approach balances survivability and offensive power, letting you handle Margit without dying instantly.

# 4-Identify which locations contain the greatest number of items, counting both distinct items and all items including duplicates.

In [0]:
%sql

SELECT
    l.location_name,
    l.region,
    COUNT(DISTINCT li.item_name) AS item_count_distinct,
    COUNT(li.item_name) AS item_count
FROM eldenringcatalog.gold.bridge_location_items li
JOIN eldenringcatalog.gold.dim_locations l
    ON li.location_id = l.location_id
GROUP BY l.location_name, l.region
ORDER BY item_count DESC
LIMIT 30;

location_name,region,item_count_distinct,item_count
Shadow Keep,Legacy Dungeons,93,100
"Leyndell, Royal Capital",Leyndell,75,84
Stormveil Castle,Legacy Dungeons,82,82
Raya Lucaria Academy,Legacy Dungeons,63,64
Belurat Tower Settlement,Legacy Dungeons,52,58
Enir-Ilim,Legacy Dungeons,53,54
Volcano Manor,Legacy Dungeons,52,53
Elphael Brace of the Haligtree,Legacy Dungeons,49,50
Castle Ensis,Legacy Dungeons,47,48
Subterranean Shunning-Grounds,Underground,41,41


Databricks visualization. Run in Databricks to view.

Based on the available data, **Shadow Keep** is the location with both the highest distinct item count and the highest total item count, making it the richest location in the game according to our dataset

# 5-Where are the closest Smithing Stones to upgrade my weapon to +3?

In [0]:
%sql
SELECT
    l.location_name,
    l.region,
    COUNT(li.item_name) AS item_count
FROM eldenringcatalog.gold.bridge_location_items li
JOIN eldenringcatalog.gold.dim_locations l
    ON li.location_id = l.location_id
WHERE region = 'Limgrave' AND item_name LIKE '%Smithing Stone%'
GROUP BY l.location_name, l.region
ORDER BY item_count DESC;

location_name,region,item_count
Mistwood,Limgrave,2
Limgrave Tunnels,Limgrave,2
Church of Dragon Communion,Limgrave,2
Highroad Cave,Limgrave,2
Stormhill,Limgrave,1
Summonwater Village,Limgrave,1
Fort Haight,Limgrave,1
Artist's Shack (Limgrave),Limgrave,1
Stormgate,Limgrave,1
Bridge of Sacrifice,Limgrave,1


The game begins in Limgrave, near the Stranded Graveyard Site of Grace. I filtered the data to include only locations within Limgrave and restricted the items to Smithing Stones. The previous query returned the count of Smithing Stones per location.

Because the dataset doesn’t provide distance information, I estimated proximity by visually examining the map. According to the data, Limgrave Tunnels contains 2 Smithing Stones and the Church of Elleh contains 1. From the map, the Church of Elleh appears to be the closest location to the starting point, so the optimal route is to go there first before heading to Limgrave Tunnels.

![](https://github.com/hakkache/EldenRingDataE2Project/blob/main/EldenRingDataE2Project/Asset/location_1.png?raw=true)

# **6-What are the top regions in the game that contain locations with multiple bosses?**

In [0]:
%sql
SELECT 
    lb.location_id,
    l.location_name,
    l.region,
    COUNT(DISTINCT lb.boss_name) AS boss_count,
    COLLECT_LIST(lb.boss_name) AS bosses
FROM eldenringcatalog.gold.bridge_location_bosses lb
JOIN eldenringcatalog.gold.dim_locations l 
    ON lb.location_id = l.location_id
GROUP BY lb.location_id, l.location_name, l.region
HAVING COUNT(DISTINCT lb.boss_name) >= 2
ORDER BY boss_count DESC

location_id,location_name,region,boss_count,bosses
217,Volcano Manor,Legacy Dungeons,6,"List(Abductor Virgins, God-Devouring Serpent, Godskin Noble, Lord of Blasphemy, Magma Wyrm, Rykard)"
229,"Leyndell, Ashen Capital",Leyndell,6,"List(Elden Beast, First Elden Lord, Godfrey, Radagon of the Golden Order, Sir Gideon Ofnir, the All-Knowing)"
230,"Leyndell, Royal Capital",Leyndell,6,"List(Divine Tower of West Altus, Leyndell Catacombs, Leyndell Colosseum, Minor Erdtree Church, Sealed Tunnel, Subterranean Shunning-Grounds)"
21,Cathedral of Manus Metyr,Scadu Altus,4,"List(Count Ymir, Mother of Fingers, Swordhand of Night Anna, Swordhand of Night Jolan)"
81,Stormveil Castle,Legacy Dungeons,4,"List(Godrick the Grafted, Grafted Scion, Lion Guardian, Ulcerated Tree Spirit)"
226,Elden Throne,Leyndell,4,"List(First Elden Lord, Godfrey, Morgott, the Omen King)"
270,Ainsel River,Underground,4,"List(Ainsel River Main, Eternal City, Nokstella, Uhl Palace Ruins)"
25,Enir-Ilim,Legacy Dungeons,3,"List(Consort of Miquella, Promised Consort Radahn, Radahn)"
70,Mistwood,Limgrave,3,"List(Ancestor Spirit, Dragonkin Soldier, Tibia Mariner)"
78,Stormhill,Limgrave,3,"List(Bell Bearing Hunter, Margit, the Fell Omen)"


# **7-Overview of Locations, Bosses, and Items by Region**

In [0]:
%sql
SELECT
    l.region,
    COUNT(DISTINCT l.location_name) AS location_count,
    COUNT(DISTINCT lb.boss_name) AS boss_count,
    COUNT(DISTINCT li.item_name) AS item_count
FROM eldenringcatalog.gold.dim_locations l
LEFT JOIN eldenringcatalog.gold.bridge_location_bosses lb
    ON l.location_id = lb.location_id
LEFT JOIN eldenringcatalog.gold.bridge_location_items li
    ON l.location_id = li.location_id
GROUP BY l.region
ORDER BY boss_count DESC;

region,location_count,boss_count,item_count
Legacy Dungeons,10,26,424
Liurnia of the Lakes,52,25,207
Limgrave,33,22,163
Altus Plateau,24,17,162
Leyndell,5,16,98
Caelid,24,15,145
Underground,16,14,171
Scadu Altus,15,13,119
Mountaintops of the Giants,17,12,94
Dragonbarrow,11,10,75


This query provides a regional summary of the game world in Elden Ring. For each region, it counts the number of distinct locations, bosses, and items, giving a comprehensive view of the game’s content distribution. By analyzing the results, we can identify which regions are densely populated with bosses and items, helping players plan their exploration routes and understand where challenges and rewards are concentrated. The data can also guide strategies for leveling up and acquiring key resources.